___

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;                        Projeto 2 - Ciência dos Dados
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                 Alexandre Cury, Guilherme Lotaif

<br>

## 1. Problema: Classificador automático de sentimento
<br>

O Classificador que este projeto terá como base é o Naive-Bayes, que essencialmente irá determinar se um pedaço de texto é relevante ou não a um assunto especifico. Iremos trabalhar com posts coletados da rede social Twitter, em forma de tweets, estes serão divididos em dois grupos: Treinamento e teste, para que possamos fazer com que nosso classificador apreenda como discernir o conteudo dos tweets.

___

## Preparando o ambiente no jupyter:

Para fazer uma análise e classificação dos nossos dados, precisamos primeiro coletar os tweets do assunto escolhido em um arquivo excel, que será usado como nossa base de dados.

In [2]:
%%capture
#nltk.download('punkt')

#Instalando o tweepy
!pip install tweepy

**Importando as bibliotecas necessárias:**

In [18]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from random import shuffle
import numpy as np
import pandas as pd
import os.path
import tweepy
import math
import json
import sys
#import nltk
import warnings

### Autenticando no  Twitter

Tendo em vista que precisamos da permissão do twitter para acessar e coletar os tweets, é necessário ter uma conta de desenvolvedor, além de solicitar as chaves de acesso. Para conseguir estas, acompanhe o [tutorial](https://developer.twitter.com/en/docs/basics/authentication/guides/access-tokens.html) o próprio twitter.

In [4]:
#Dados de autenticação do twitter:
#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

### Escolha do produto e coleta das mensagens

Agora é feita a seleção do produto que iremos análisar, e para coletar os tweets para o nosso projeto, é preciso especificar a quantidade de posts que serão importados `(quanto maior esta quantidade for, mais preciso será o nosso classificador)`.

In [ ]:
#Produto escolhido:
produto = 'Airpods'
#Quantidade mínima de mensagens capturadas:
n = 1000
#Quantidade mínima de mensagens para a base de treinamento:
t = 750
#Filtro de língua, escolha uma na tabela ISO 639-1:
lang = 'en'

**Capturando os dados do twitter:**

In [ ]:
#Cria um objeto para a captura:
api = tweepy.API(auth)
#Inicia a captura, para mais detalhes: ver a documentação do tweepy:
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés:
shuffle(msgs)

**Salvando os dados em uma planilha Excel:**

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

<br>

___
### Importando e limpando os tweets

Esta etapa é muito importante, porque a maioria dos posts acabam saindo com muita "sujeira", esta pode ser desde vírgulas até parênteses, que podem atrapalhar o classificador. Outra ferramenta de limpeza que usaremos será para manter somente o radical da palavra, assim vamos diminuir a variação de palavras com o mesmo radical mas complementos diferentes.

**Abrindo o arquivo:**<br>
Este já possui uma classificação feita manualmente.

In [5]:
#Atribuindo o arquivo de treinamento com os tweets a uma variável:
df_train = pd.read_excel('Airpod.xlsx',0)
#Atribuindo o arquivo de teste com os tweets a uma variável:
df_test = pd.read_excel('Airpod.xlsx',1)

In [6]:
#Mostrando uma amostra de um dos dataframes:
df_train.sample(2)

,Treinamento,B1
226,rt @itsghibli: spirited away airpod case 💞☁️ h...,0
201,"left airpod in hers, right in his, inside targ...",1


...

#### Vamos remover a coluna de comparação:

Para conseguirmos medir a competência do nosso classificador, foi feita manualmente uma coluna extra com a relevância de cada tweet, que sera usada para comparar os resultados obtidos no final deste projeto. Porém ela precisa ser removida para que possamos analisar somente o conteudo dos posts.

In [7]:
#Removendo a coluna "B1" do dataframe de treino:
df_train_1 = df_train.drop('B1',axis=1)
#Removendo a coluna "B1" do dataframe de teste:
df_test_1 = df_test.drop('B1',axis=1)

...

#### Vamos criar uma função para a limpeza dos tweets:

Esta função será usada mais para frente, ao separarmos os tweets em duas categorias diferentes: Relevantes e Irrelevantes, com o intuito de limpar as frases de cada post e e separar as palavras, para facilitar que o classificador apreenda as distinguir os dois tipos de tweet.

In [20]:
#Vamos criar uma função que irá substituir os caracteres que trapalham o classificador:
def Cleaner (Tweets):
    #Os elementos na primeira posição do replace serão substituidos por espaços:
    Tweets = Tweets.replace("*","")
    Tweets = Tweets.replace("!","")
    Tweets = Tweets.replace("@","")
    Tweets = Tweets.replace("#","")
    Tweets = Tweets.replace(".","")
    Tweets = Tweets.replace(":","")
    Tweets = Tweets.replace(")","")
    Tweets = Tweets.replace("(","")
    Tweets = Tweets.replace("/","")
    Tweets = Tweets.replace('"',"")
    Tweets = Tweets.replace("[","")
    Tweets = Tweets.replace(']',"")
    Tweets = Tweets.replace("\ ","")
    Tweets = Tweets.replace("rt","")
    Tweets = Tweets.replace("&","")
    Tweets = Tweets.replace("-","")
    Tweets = Tweets.replace("_","")
    Tweets = Tweets.replace("+","")
    Tweets = Tweets.replace("=","")
    Tweets = Tweets.replace("'","")
    Tweets = Tweets.replace("?","")
    Tweets = Tweets.replace(",","")
    
    #Vamos retirar os links:
    for Word in Tweets:
        if 'https' in Word:
            Tweets.remove(Word)
    
    #---------------------------------------------------------------
    #Utilização do stemmer de palavras: que elimina os complementos e deixa somente o radical das palavras
    
    Post = word_tokenize(Tweets) 
    Tweets_steam = [ps.stem(word) for word in Post]
    #---------------------------------------------------------------
    
    
    #Aproveitando a nossa função ja vamos separar todas as palavras:
    Tweets_final = Tweets_steam

    return Tweets_final

In [19]:
ps = PorterStemmer()

<br>

___
### Separando os Tweets relevantes dos irrelevantes:

Para termos mais noção se o nosso dataframe possui uma quantidade boa de marcadores da relevância dos tweets para "ensinar" o classificador.


#### Descobrindo quantos tweets relevantes o dataframe possui:

In [9]:
#Atribuindo a quantidade de tweets relevantes e irrelevantes no dataframe:
Relevant, Irrelevant, Really_relevant = (df_train["B1"]).value_counts()
#Descobrindo a quantidade total de tweets no dataframe:
Total_train = df_train["B1"].value_counts().sum()
#Criando uma lista com as informações dos tweets:
Tweets_rel_ire_1 =["Tweets", Irrelevant, Relevant, Really_relevant, "1.00"]
#Criando uma lista com as informações pelo total dos tweets:
TOTAL_tweets_1 = ["TOTAL", "{:.2f}".format(Irrelevant/Total_train), "{:.2f}".format(Relevant/Total_train),
                  "{:.2f}".format(Really_relevant/Total_train), "1.00"]

In [10]:
#Atribuindo os valores à lista com todos dados:
data_rel = [Tweets_rel_ire_1, TOTAL_tweets_1]
#Transformando essa lista em um dataframe:
data_relevance = (pd.DataFrame(data_rel, columns=[" ","Irrelevantes","Relevantes","Muito Relevantes","TOTAL"])).set_index(' ')
#Plotando os dados:
data_relevance

,Irrelevantes,Relevantes,Muito Relevantes,TOTAL
,,,,
Tweets,188,233,79,1.00
TOTAL,0.38,0.47,0.16,1.00


#### Com a tabela acima conseguimos saber algumas probabilidades quanto aos TWEETS:
1. A probabilidade de um tweet ser irrelevante:

    $P(Irrelevante)$

In [11]:
Irrelevant_tot_prob = Irrelevant/Total_train
print("{:.2f}%".format((Irrelevant_tot_prob)*100))

37.60%


2. A probabilidade de um tweet ser relevante:

    $P(Relevante)$

In [12]:
Relevant_tot_prob = Relevant/Total_train
print("{:.2f}%".format((Relevant_tot_prob)*100))

46.60%


3. A probabilidade de um tweet ser Muito relevante:

    $P(Muito Relevante)$

In [13]:
Really_relevant_tot_prob = Really_relevant/Total_train
print("{:.2f}%".format((Really_relevant_tot_prob)*100))

15.80%


...
<br>


Vamos criar um contador para as variaveis relevantes, assim como um dicionário para contar as repetições:

In [14]:
#Vamos criar contadores para o total de tweets relevantes e irrelevantes:
Really_relevant_count, Relevant_count, Irrelevant_count = 0,0,0
#Vamos criar dicionários para os Tweets relevantes e irrelevantes:
Really_relevant_dict, Relevant_dict, Irrelevant_dict = {},{},{}

In [21]:
#Vamos criar um loop para rodar todos os tweets e selecinar só os irrelevantes:
for Tweet in df_train.Treinamento[df_train.B1 == 0]:
    #Vamos atribuir todos os tweets irrelevantes à uma váriavel:
    Irrelevant_list = Cleaner(Tweet)
    #Vamos criar um loop secundário para rodar as palavras de cada tweet:
    for Word in Irrelevant_list:
        if Word in Irrelevant_dict:
            Irrelevant_count += 1
            Irrelevant_dict[Word] += 1
        else:
            Irrelevant_count += 1
            Irrelevant_dict[Word] = 1

#Vamos criar um loop para rodar todos os tweets e selecinar só os relevantes:
for Tweet in df_train.Treinamento[df_train.B1 == 1]:
    #Vamos atribuir todos os tweets relevantes à uma váriavel:
    Relevant_list = Cleaner(Tweet)
    #Vamos criar um loop secundário para rodar as palavras de cada tweet:
    for Word in Relevant_list:
        if Word in Relevant_dict:
            Relevant_count += 1
            Relevant_dict[Word] += 1
        else:
            Relevant_count += 1
            Relevant_dict[Word] = 1
            
#Vamos criar um loop para rodar todos os tweets e selecinar só os relevantes:
for Tweet in df_train.Treinamento[df_train.B1 == 2]:
    #Vamos atribuir todos os tweets relevantes à uma váriavel:
    Really_relevant_list = Cleaner(Tweet)
    #Vamos criar um loop secundário para rodar as palavras de cada tweet:
    for Word in Really_relevant_list:
        if Word in Really_relevant_dict:
            Really_relevant_count += 1
            Really_relevant_dict[Word] += 1
        else:
            Really_relevant_count += 1
            Really_relevant_dict[Word] = 1

...
<br>

#### Vamos análisar um pouco os dados obtidos acima:

Ja separamos as palavras presentes nos dois tipos de tweets, agora vamos mostrar a comparação das caracteristicas dos dois grupos:

In [22]:
#Atribuindo a quantidade de palavras repetidas nos tweets relevantes e irrelevantes:
Irrelevant_rep, Relevant_rep, Really_relevant_rep = (len(Irrelevant_dict)),(len(Relevant_dict)),(len(Really_relevant_dict))
#Atribuindo a quantidade de palavras NÃO repetidas nos tweets irrelevantes:
Irrelevant_not_rep = (Irrelevant_count-len(Irrelevant_dict))
#Atribuindo a quantidade de palavras NÃO repetidas nos tweets relevantes e muito relevantes:
Relevant_not_rep, Really_relevant_not_rep = (Relevant_count-len(Relevant_dict)),(Really_relevant_count-len(Really_relevant_dict))
#Descobrindo a quantidade total de palavras nos tweets:
Words_total = Really_relevant_count + Relevant_count + Irrelevant_count
#Vamos criar uma lista para atribuir os valores das palavras com repetição:
Words_repetition = [" Presença de Repetições", Irrelevant_rep, Relevant_rep, Really_relevant_rep ,
                    "{:.2f}".format((Irrelevant_rep+Relevant_rep+Really_relevant_rep)/Words_total)]
#Vamos criar uma lista para atribuir os valores das palavras sem repetição:
Words_not_repetition = ["Ausência de Repetições", Irrelevant_not_rep, Relevant_not_rep, Really_relevant_not_rep,
                        "{:.2f}".format((Irrelevant_not_rep+Relevant_not_rep+Really_relevant_not_rep)/Words_total)]
#Vamos criar uma lista para atribuir os valores das palavras pelo total:
TOTAL_tweets_2 = ["TOTAL", "{:.2f}".format((Irrelevant_not_rep+Irrelevant_rep)/Words_total),
                  "{:.2f}".format((Relevant_not_rep+Relevant_rep)/Words_total),
                  "{:.2f}".format((Really_relevant_not_rep+Really_relevant_rep)/Words_total),"1.00"]

In [23]:
#Atribuindo os valores à lista com todos dados:
data_rel = [Words_repetition, Words_not_repetition, TOTAL_tweets_2]
#Transformando essa lista em um dataframe:
data_relevance = (pd.DataFrame(data_rel, columns=["Palavras","Irrelevantes","Relevantes","Muito Relevantes",
                                                  "TOTAL"])).set_index('Palavras')
#Plotando os dados:
data_relevance

,Irrelevantes,Relevantes,Muito Relevantes,TOTAL
Palavras,,,,
Presença de Repetições,243,1068,534,0.27
Ausência de Repetições,1341,2750,850,0.73
TOTAL,0.23,0.56,0.20,1.00


<br>

___
### Montando o Classificador Naive-Bayes

<br>

**Um pouco sobre a funcionalidade de um classificador:** A partir dos dados de treinamento, o classificador "apreende" quais palavras são mais prováveis de acontecer em tal nível de relevância. Portanto, quando usamos os dados de teste para comprovar, mais tarde, podemos perceber que podem haver certas inconsistencias, devido ao fato da nosa base de tweets ser pequena, e que o classificador analisa cada palavra de maneira indivídual.
<br><br>

...

**Agora vamos usar o classificador:** 

<br>
Mostrando como lidar com probabilidade condicional através do Teorema de Bayes:


$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$
<br>
Se substituirmos as devidas variáveis, para adequar ao nosso problema:

- `Para sabermos a probabilidade de ser relevantes`<br>
$$P(RELEVANTE|Tweet) = \frac{P(Tweet|RELEVANTE) \cdot P(RELEVANTE)}{P(Tweet)}$$
<br>
<br>
- `Para sabermos a probabilidade de ser irrelevantes`
$$P(IRRELEVANTE|Tweet) = \frac{P(Tweet|IRRELEVANTE) \cdot P(IRRELEVANTE)}{P(Tweet)}$$
<br>

Com o objetivo de descobrir se é relevante ou não, é cabivel dividir um pelo outro:<br>
<br>

$$\frac {P(RELEVANTE|Tweet)}{P(IRRELEVANTE|Tweet)}=\frac {P(Tweet|RELEVANTE)\cdot P(RELEVANTE)}{P(Tweet|IRRELEVANTE)\cdot P(RELEVANTE)}$$<br><br>

...
<br><br>

Abaixo vamos fazer uma função que nos permitirá calcular quantas vezes cada palavra pertencente ao tweet escolhido pelo usuário aparece em cada dicionário: dos irrelevante, relevantes e dos muito relevantes. apos isso ela faz uma conta da probabilidade do tweet que apareceu seja de ta grupo, uma vez que dividimos a quantidade de repetição pelo total:

In [26]:
#Vamos elaborar uma função que vai calcular a probabilidade do Tweet ser relevante ou não:
def Probability_estimator(Dictionary, TWEET, Words_quantity, Probability):
    #Vamos atribuir todos os tweets limpos à uma váriavel:
    ALL_Cleaned_tweet = Cleaner(TWEET)

    #Vamos checar as palavras do dicionário e suas repetições através de um loop:
    for Words in ALL_Cleaned_tweet:

        Amount = 1
        if Words in Dictionary:
            Amount += Dictionary[Words]
        Probability += Amount/(Words_quantity + Words_total)

    #Retorna a probablidade:
    return Probability

...

<br>

A função a seguir irá destinguir qual probabilidade é maior, e vai retornar o resultado que vencer as restrições:

In [27]:
#Vamos criar uma  função que vai comparar os resultados e nos dizer qual é mais provável:
def Probability_Inspector(Irrelevant_prob, Relevant_prob, Really_relevannt_prob):
    if Irrelevant_prob > Relevant_prob and Irrelevant_prob > Really_relevant_prob:
        return "IRRELEVANTE"
    
    elif Irrelevant_prob < Relevant_prob and Really_relevant_prob < Relevant_prob:
        return "RELEVANTE"
    
    elif Irrelevant_prob < Really_relevant_prob and Relevant_prob < Really_relevant_prob:
        return "MUITO RELEVANTE"
    
    else:
        return "EM CIMA DO MURO"

...

<br>

**Espaço para inserir o tweet a ser classificado:**

In [29]:
#Vamos pedir um tweet para ser classificado:
Tweet_input = input("What's happening: ")

What's happening: big airpods


In [30]:
#Vamos calcular a probabilidade de ser irrelevante:
Irrelevant_prob = Probability_estimator(Irrelevant_dict, Tweet_input, Irrelevant_count, Irrelevant_tot_prob)

In [31]:
#Vamos calcular a probabilidade de ser relevante:
Relevant_prob = Probability_estimator(Relevant_dict, Tweet_input, Relevant_count, Relevant_tot_prob)

In [32]:
#Vamos calcular a probabilidade de ser muito relevante:
Really_relevant_prob = Probability_estimator(Really_relevant_dict, Tweet_input, Really_relevant_count, Really_relevant_tot_prob)

In [33]:
#Vamos utilizar a função criada acima para descobrir a classe correta:
Final_decision = Probability_Inspector(Irrelevant_prob, Relevant_prob, Really_relevant_prob)

In [34]:
Final_decision

'RELEVANTE'

<br>

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [40]:
Results = []

for Tweet_msg in df_test["B1"]:
    Irrelevant_prob_test = Probability_estimator(Irrelevant_dict, Tweet_input, Irrelevant_count, Irrelevant_tot_prob)
    Relevant_prob_test = Probability_estimator(Relevant_dict, Tweet_input, Relevant_count, Relevant_tot_prob)
    Really_relevant_prob_test = Probability_estimator(Really_relevant_dict, Tweet_input, Really_relevant_count, Really_relevant_tot_prob)
    Results.append(Probability_Inspector(Irrelevant_prob, Relevant_prob, Really_relevant_prob))

df_test['Likely'] = Results

In [41]:
#Vamos criar as variáveis para atribuir os valores desejados:
Positive_true = 0
Positive_false = 0
Negative_true = 0
Negative_false = 0

In [42]:
#Vamos usar o codigo zip para percorrer:
for likely_test, b1_test in zip(df_test["Likely"],df_test["B1"]):
    if likely_test == "MUITO RELEVANTE" and b1_test == "MUITO RELEVANTE":
        Positive_true += 1
    elif likely_test == "RELEVANTE" and b1_test == "RELEVANTE":
        Positive_true += 1
    elif likely_test == "MUITO RELEVANTE" and b1_test != "MUITO RELEVANTE":
        Positive_false += 1
    elif likely_test == "RELEVANTE" and b1_test != "RELEVANTE":
        Positive_false += 1
    elif likely_test == "IRRELEVANTE" and b1_test == "IRRELEVANTE":
        Negative_true += 1    
    else:
        Negative_false += 1

___
### Concluindo

Este classificador de relevancia de tweets, está de acordo com o teorema de bayes. Como sua base de dados é pequena e estamos trabalhando com  3 classificações, entao sua eficacea é pequena, porém ele tem um bom potencial.  Uma outra possível aplicação para o classificador de bayes é um detector de emails spam.

# Referências:

- https://github.com/GuilhermeLotaif/SPAM/blob/master/Projeto%202/Projeto2%20Layout.ipynb

- https://www.geeksforgeeks.org/python-stemming-words-with-nltk/

- https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/